In [ ]:
import os

import trimesh
import meshplot as mp

import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp

import warnings
warnings.filterwarnings('ignore')


In [ ]:
RES = './meshes/'

mesh_fp = RES + 'cow_manifold2.obj'
assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp 
cow = trimesh.load_mesh(mesh_fp) 

mesh_fp = RES + 'human.obj'
assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp 
human = trimesh.load_mesh(mesh_fp) 

## Laplace-Beltrami Operator

In [ ]:
def Cotangent_Laplce_Matrix(mesh: trimesh.base.Trimesh):
    
    '''
        Cotangent Laplace Beltrami 
    '''

    vertices = mesh.vertices

    faces = mesh.faces
    angles = mesh.face_angles
    areas = mesh.area_faces

    A = np.zeros(len(vertices))
    for i in range(len(faces)):
        face = faces[i]
        area = areas[i]
        A[face] += area / 3

    L = np.zeros(shape=(len(vertices), len(vertices)))
    for i in range(len(faces)):
        face = faces[i]
        angle = angles[i]

        v0, v1, v2 = face
        for j, (m, n) in enumerate([(v1, v2), (v0, v2), (v0, v1)]):
            cot = (1 / np.tan(angle[j])) * (1 / (A[m] + A[n]))
            L[m, n] += cot
            L[n, m] += cot
            L[n, n] -= cot
            L[m, m] -= cot

    return sp.csr_matrix(L)


def Mesh_Laplace_Matrix(mesh : trimesh.Trimesh):
    '''
        Mesh Laplace Matrix 
    '''

    vertices = mesh.vertices
    edges = mesh.edges_unique
    edge_length = mesh.edges_unique_length

    h = np.mean(edge_length)
    f = 1.0 / (4 * np.pi * h * h)
    w = f * np.exp(-edge_length**2/(4.0*h))

    L = np.zeros(shape=(len(vertices), len(vertices)))
    for idx, (i, j) in enumerate(edges):
        L[i, i] += w[idx]
        L[j, j] += w[idx]
        L[i, j] -= w[idx]
        L[j, i] -= w[idx]

    return sp.csr_matrix(L)

## Heat Diffusion

In [ ]:
def Find_Boundary_Vertices(mesh):
    '''
        Handle Boundary cases
    '''
    edge_count = {}
    
    for face in mesh.faces:
        edges = [(min(face[i], face[j]), max(face[i], face[j])) for i, j in [(0, 1), (1, 2), (2, 0)]]
        for edge in edges:
            if edge in edge_count:
                edge_count[edge] += 1
            else:
                edge_count[edge] = 1
    
    # Find vertices that are part of boundary edges
    boundary_vertices_set = set()
    for edge, count in edge_count.items():
        if count == 1:  # Edge is a boundary edge
            boundary_vertices_set.update(edge)
    
    boundary_vertices = list(boundary_vertices_set)
    
    return boundary_vertices

In [ ]:
def get_heat(eigval, eigvec, t, ver_idx, mesh, heatValue=100.0):
    '''
        Use heat operator to get heat
    '''

    I = np.zeros(eigvec.shape[0])
    I[ver_idx] = heatValue
    coeffs = (I @ eigvec) * np.exp(-eigval * t)
    heat = eigvec @ coeffs.T

    boundary_vertices = Find_Boundary_Vertices(mesh)
    if len(boundary_vertices) !=0:
    # # Set the function value to 0 at each boundary vertex
        for vertex in boundary_vertices:
            heat[vertex] = 0
            
    return heat

In [ ]:
def Heat_Operator(mesh, k, heat_value, vertex_index, t_index, vertex_labels = ["Front Right Foot", "Front Left Foot", "Back Right Foot", "Back Left Foot", "Head"], method = Cotangent_Laplce_Matrix, plot_graph=True, plot_mesh=True): 
    '''
        Use heat operator to plot graph heat difference as times passes
    '''

    L = method(mesh)
    e_values, e_vectors = sp.linalg.eigs(L, k=k, which='SM')

    num_interval = 100

    T = np.linspace(4 * np.log(10) / e_values[-1], 4 * np.log(10) / e_values[1], num_interval, endpoint=True)
    heat = np.zeros(shape=(num_interval, len(vertex_index)))
    scaled_heat = np.zeros_like(heat)

    for i in range(num_interval):
        h = get_heat(e_values, e_vectors, T[i], vertex_index, mesh, heatValue=heat_value)
        for j in range(len(vertex_index)):
            heat[i][j] = h[vertex_index[j]].real.item()
    
    for i in range(num_interval):
        integral_approx = np.sum(heat[i, :])
        scaled_heat[i, :] = heat[i, :] / integral_approx if integral_approx != 0 else heat[i, :]

    if plot_mesh:
        h = get_heat(e_values, e_vectors, t_index, vertex_index, mesh, heatValue=heat_value)
        print("Heat Distribution at time {0} with eigenfunctions of {1}".format(t_index, k))
        mp.plot(mesh.vertices, mesh.faces, h.real)

    if plot_graph:
        for i in range(len(vertex_index)):
            plt.plot(np.log10(T), scaled_heat[:,i], label = vertex_labels[i])
        plt.legend()
        plt.show()

    return scaled_heat

In [ ]:
# Cow
cow_heat = Heat_Operator(mesh=cow, k=300, heat_value=100, t_index=0.5, vertex_index=[2483, 2406, 2522, 2445, 829], method=Mesh_Laplace_Matrix)

In [ ]:
# Human
human_heat = Heat_Operator(mesh=human, k=300, heat_value=100, t_index=3, vertex_index=[4316, 4504, 66, 53, 9525], vertex_labels = ["Right Hand", "Left Hand", "Right Foot", "Left Foot", "Head"], method=Mesh_Laplace_Matrix)

## Heat Kernel Signature

In [ ]:
def get_hks(eigenval, eigenvec, t):
    '''
        Use heat kernel signiture to get heat
    '''
    hks = (eigenvec**2) * np.exp(eigenval * t * -1)
    return np.sum(hks, axis=1)

In [ ]:
def Heat_Kernel_Signature(mesh, vertex_index, k=300, t_index=0.5, vertex_labels = ["Front Right Foot", "Front Left Foot", "Back Right Foot", "Back Left Foot", "Head"], method = Cotangent_Laplce_Matrix, plot_graph=True, plot_mesh=True):
    '''
        Use HKS to plot graph heat difference as times passes
    '''

    L = method(mesh)
    e_values, e_vectors = sp.linalg.eigs(L, k=k, which='SM')

    num_interval = 100

    T = np.linspace(4 * np.log(10) / e_values[-1], 4 * np.log(10) / e_values[1], num_interval, endpoint=True)
    heat = np.zeros(shape=(num_interval, len(vertex_index)))
    scaled_heat = np.zeros_like(heat)

    for i in range(num_interval):
        h = get_hks(e_values, e_vectors, T[i])
        heat[i] = h[vertex_index].real

    for i in range(num_interval):
        integral_approx = np.sum(heat[i, :])
        scaled_heat[i, :] = heat[i, :] / integral_approx if integral_approx != 0 else heat[i, :]

    if plot_mesh:
        h = get_hks(e_values, e_vectors, t_index)
        print("Heat Distribution at time {0} with eigenfunctions of {1}".format(t_index, k))
        mp.plot(mesh.vertices, mesh.faces, h.real)

    if plot_graph:
        for i in range(len(vertex_index)):
            plt.plot(np.log10(T), scaled_heat[:,i], label = vertex_labels[i])
        plt.legend()
        plt.show()

    return scaled_heat

In [ ]:
# Cow
cow_hks = Heat_Kernel_Signature(cow, t_index=0.5, vertex_index=[2483, 2406, 2522, 2445, 829], method=Mesh_Laplace_Matrix)

In [ ]:
# Human
human_hks = Heat_Kernel_Signature(human, t_index=3, vertex_index=[4316, 4504, 66, 53, 9525], vertex_labels = ["Right Hand", "Left Hand", "Right Foot", "Left Foot", "Head"], method=Mesh_Laplace_Matrix)

In [ ]:
def Heat_Kernel_Signature_Dis(mesh, t, vertex_index, method=Cotangent_Laplce_Matrix):
    '''
        HKS scaled difference
    '''

    L = method(mesh)
    e_values, e_vectors = sp.linalg.eigs(L, k=300, which='SM')

    vertices = mesh.vertices
    num_interval = 100
    
    T = np.linspace(t[0], t[1], num_interval, endpoint=True)
    heat = np.zeros(shape=(num_interval, len(vertices)))
    delta_log_t = np.diff(T, prepend=T[0]) 
    
    diff_set = np.zeros(len(vertices))

    for i in range(num_interval):
        h = get_hks(e_values, e_vectors, T[i])
        for j in range(len(vertices)):
            heat[i][j] = h[j].real.item()
        normalise_factor = np.sum(np.float64(h.real))
        heat[i] /= normalise_factor

    for j in range(len(vertices)):
        if j!=vertex_index:
            bong = (heat[:, vertex_index]-heat[:, j])** 2
            diff = np.sqrt(np.sum(bong * delta_log_t, axis =0))
            diff_set[j] = diff    

    mp.plot(mesh.vertices, mesh.faces, diff_set)

    return diff_set

In [ ]:
# Cow
cow_diff = Heat_Kernel_Signature_Dis(cow, t=[1,2], vertex_index=2483, method = Mesh_Laplace_Matrix)

In [ ]:
# Human
human_diff = Heat_Kernel_Signature_Dis(human, t=[0.01, 1000000], vertex_index=4316, method = Mesh_Laplace_Matrix)

## Extension

**Data Importing**

In [ ]:
RES = './SHREC15/train'

keys = ["santa", "horse", "dog", "bird", "laptop", "female", "female", "male", "child", "male"]
training_set = {}
for key in keys:
    training_set[key] = []

n = 10    
for id in range(len(os.listdir(RES))):
    class_name = keys[id // n]
    if class_name == "child":
        continue
    f = str(id) + ".obj"
    
    training_set[class_name].append(trimesh.load_mesh(os.path.join(RES, f))) 

**Bag of Features**

In [ ]:
def BagOfFeature(mesh, num_clusters, alpha, t0, k=300, method=Cotangent_Laplce_Matrix):

    '''
        Return Bag of Featrues of mesh
    '''

    from sklearn.cluster import KMeans
    from scipy.spatial.distance import pdist

    L = method(mesh)
    e_values, e_vectors = sp.linalg.eigs(L, k=k, which='SM')

    vertices = mesh.vertices
    num_interval = 10
    
    # Time Interval
    T = np.array([t0 * alpha**i for i in range(num_interval)])
    heat = np.zeros(shape=(num_interval, len(vertices)))

    # Heat Feature Description
    for i in range(num_interval):
        h = get_hks(e_values, e_vectors, T[i])
        heat[i,:] = h.real

        heat[i,:] /= np.linalg.norm(heat[i,:])
    heat = heat.T

    #Kmean
    kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
    kmeans.fit(heat)
    centers = kmeans.cluster_centers_
    
    # Calculate pairwise distances between centers and find the median
    distances = pdist(centers)
    sigma = np.median(distances)

    # Calculate the Bag of Features histogram
    f_distribution = np.zeros((len(vertices), num_clusters))
    for i in range(len(vertices)):
        for j in range(num_clusters):
            f_distribution[i][j] = np.exp(-np.linalg.norm(heat[i, :] - centers[j]) / (2 * sigma**2))
            
        constraint = np.sum(f_distribution[i, :])
        if constraint != 0:
            f_distribution[i, :] /= constraint
        
    BoF = np.sum(f_distribution, axis=0)
    
    return BoF

In [ ]:
def BoF_Data_Class(dataset, k, alpha, t0):
    '''
       Compute Bag of features of dataset at once
    '''

    keys = dataset.keys()
    class_names = list(keys)

    y = []
    x = []
    for i, name in enumerate(class_names):

        meshes = dataset[name]

        for m in range(len(meshes)):
            BoF = BagOfFeature(meshes[m], k, alpha, t0, method=Mesh_Laplace_Matrix)
            if len(BoF) == k:
                x.append(BoF)
                y.append(i)
                print("Finished! {} - {}".format(name, m))

    return x, y


In [ ]:
data, labels = BoF_Data_Class(training_set, k=10, alpha=1.32, t0=0.1)

In [ ]:
# Save Bag of features and lables
np.save('./reference_BoF/BoF.npy', data)
np.save('./reference_BoF/labels.npy', labels)

**K-Nearest Neighbors**

In [ ]:
# Load Bag of features and lables
data = np.load('./reference_BoF/BoF.npy')
labels = np.load('./reference_BoF/labels.npy')

In [ ]:
def BoF_KNN(train, label, test):

    '''
        Perform KNN
    '''

    from sklearn.neighbors import KNeighborsClassifier

    # Create KNN classifier with n_neighbors and p=1 for L1 distance
    knn = KNeighborsClassifier(n_neighbors=11, p=1, metric='minkowski')

    # Fit the classifier to the training data
    knn.fit(train, label)

    probabilities = knn.predict_proba(test)

    return probabilities


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

data = np.array(data)  
labels = np.array(labels)  

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=101)

for train_index, test_index in sss.split(data, labels):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

probabilities = BoF_KNN(X_train, y_train, X_test)  

## Evaluation

**PR Curve**

In [ ]:
def PR_curve(probability, gt, class_names=None):
    '''
        Plot PR curve
    '''

    from sklearn.metrics import precision_recall_curve, average_precision_score

    n_classes = len(np.unique(gt))

    Y_test = []
    for i in range(len(gt)):
        label = np.zeros(n_classes)
        label[gt[i]] = 1
        Y_test.append(label)
    Y_test = np.array(Y_test)

    probability = np.array(probability)

    precision = dict()
    recall = dict()
    average_precision = dict()

    plt.figure()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i], probability[:, i])
        average_precision[i] = average_precision_score(Y_test[:, i], probability[:, i])

        if class_names:
            plt.step(recall[i], precision[i], where='post', label=class_names[i])
        else:
            plt.step(recall[i], precision[i], where='post', label='Class {0}'.format(i))
        
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve')
    plt.legend(loc="upper right")
    plt.show()

    return precision, recall, average_precision

In [ ]:
_, _, _ = PR_curve(probabilities, y_test, class_names=["santa", "horse", "dog", "bird", "laptop", "female", "male"])

**MDS Graph**

In [ ]:
def MDS_Graph(meshes, vertex_index, method=Cotangent_Laplce_Matrix):

    '''
        Plot MDS Graph
    '''
    
    from sklearn.manifold import MDS

    data = []
    for i, mesh in enumerate(meshes):
        heat = Heat_Kernel_Signature(mesh, vertex_index, method=method, plot_mesh=False, plot_graph=False)
        if i == 0:
            data = heat
        else:
            data = np.hstack((data, heat))

    # Normalize data
    for i in range(data.shape[0]):
        data[i, :] /= np.sum(data[i, :])

    mds = MDS(n_components=2)
    embedding = mds.fit_transform(data.T)

    # Generate colors and labels
    colors = [np.random.random((1, 3)) for _ in range(len(meshes))]
    labels = [f'human{i+1}' for i in range(len(meshes))]

    plt.figure(figsize=(8, 6))
    # Plot each mesh with its respective color and label
    for i, color in enumerate(colors):
        indices = range(i * len(vertex_index), (i + 1) * len(vertex_index))
        plt.scatter(embedding[indices, 0], embedding[indices, 1], c=color, label=labels[i])

    vertex_labels = ["Right Hand", "Left Hand", "Right Foot", "Left Foot", "Head"]
    for i, text in enumerate(vertex_labels):
        plt.annotate(text, (embedding[i, 0], embedding[i,1]))

    plt.xlabel('MDS Dimension 1')
    plt.ylabel('MDS Dimension 2')
    plt.title('MDS Plot of Time vs. Vertex')
    plt.grid(True)
    plt.legend()  
    plt.show()



In [ ]:
human1 = trimesh.load_mesh('./meshes/human1.obj') 
human2 = trimesh.load_mesh('./meshes/human2.obj') 
human3 = trimesh.load_mesh('./meshes/human3.obj') 

meshes = [human1, human2, human3]

In [ ]:
MDS_Graph(meshes, vertex_index=[4316, 4504, 66, 53, 9525], method = Mesh_Laplace_Matrix)